In [ ]:
import pdb
import pickle
import string

import time

import nltk
import numpy as np
from nltk.corpus import stopwords, twitter_samples

from utils import (cosine_similarity, get_dict,
                   process_tweet)
from os import getcwd



In [2]:
filePath = f"{getcwd()}/tmp2/"
nltk.data.path.append(filePath)

### Getting the word embeddings Data for English and French Words

In [3]:
en_embeddings_subset = pickle.load(open("./data/en_embeddings.p", "rb"))
fr_embeddings_subset = pickle.load(open("./data/fr_embeddings.p", "rb"))

#### Load two dictionaries mapping the English to French words
* A training dictionary
* and a testing dictionary.

In [4]:
en_fr_train = get_dict('./data/en-fr.train.txt')
print('The length of the English to French training dictionary is', len(en_fr_train))
en_fr_test = get_dict('./data/en-fr.test.txt')
print('The length of the English to French test dictionary is', len(en_fr_test))

c:\Users\hp\Desktop\MLAIM\projects\LSH_for_Github\utils.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  en = my_file.loc[i][0]
c:\Users\hp\Desktop\MLAIM\projects\LSH_for_Github\utils.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fr = my_file.loc[i][1]


The length of the English to French training dictionary is 5000
The length of the English to French test dictionary is 1500


c:\Users\hp\Desktop\MLAIM\projects\LSH_for_Github\utils.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  en = my_file.loc[i][0]
c:\Users\hp\Desktop\MLAIM\projects\LSH_for_Github\utils.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fr = my_file.loc[i][1]


#### Generate Embedding and Transform Matrices

In [5]:
def get_matrices(en_fr, french_vecs, english_vecs):
    """
    Creates matrices of word embeddings for English and French words that are mapped to each other.
    
    Inputs:
        en_fr: Dictionary mapping English words to French words.
        french_vecs: Dictionary of French word embeddings.
        english_vecs: Dictionary of English word embeddings.
    
    Outputs: 
        X: Matrix with each row being the embedding of an English word. Shape is (number_of_words, embedding_size).
        Y: Matrix with each row being the embedding of the corresponding French word. Shape matches X.
    
    Note:
        This function does not compute or return a projection matrix.
    """

   
    X_l = list()
    Y_l = list()

    english_set = english_vecs.keys()

    french_set = french_vecs.keys()

    french_words = set(en_fr.values())

    for en_word, fr_word in en_fr.items():

        if fr_word in french_set and en_word in english_set:

            en_vec = english_vecs[en_word]

            fr_vec = french_vecs[fr_word]

            X_l.append(en_vec)

            Y_l.append(fr_vec)

    X = np.vstack(X_l)

    Y = np.vstack(Y_l)

    return X, Y

In [6]:
# getting the training set:
X_train, Y_train = get_matrices(
    en_fr_train, fr_embeddings_subset, en_embeddings_subset)

In [10]:
X_train, Y_train

(array([[ 0.08007812,  0.10498047,  0.04980469, ...,  0.00366211,
          0.04760742, -0.06884766],
        [ 0.02600098, -0.00189209,  0.18554688, ..., -0.12158203,
          0.22167969, -0.02197266],
        [-0.01177979, -0.04736328,  0.04467773, ...,  0.07128906,
         -0.03491211,  0.02416992],
        ...,
        [-0.17089844,  0.17871094, -0.06494141, ..., -0.10644531,
         -0.31640625, -0.09326172],
        [-0.21875   ,  0.09179688,  0.03637695, ..., -0.015625  ,
         -0.27148438,  0.14941406],
        [-0.00418091,  0.0703125 , -0.04516602, ..., -0.16015625,
          0.09326172, -0.15039062]], dtype=float32),
 array([[-0.0061825 , -0.00094387, -0.00882648, ...,  0.111644  ,
         -0.0503964 , -0.0603421 ],
        [-0.0341354 ,  0.042414  , -0.0656882 , ..., -0.0539992 ,
          0.0371097 , -0.0433599 ],
        [ 0.0426481 ,  0.0395683 , -0.00825683, ...,  0.0295259 ,
          0.0713421 ,  0.0626402 ],
        ...,
        [ 0.0903279 , -0.108363  , -0.0

#### Computing the Loss

In [11]:
def compute_loss(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        L: a matrix of dimension (m,n) - the value of the loss function for given X, Y and R.
    '''
    m = X.shape[0]
    
    diff = np.dot(X,R)-Y

    diff_squared = diff**2

    sum_diff_squared = np.sum(diff_squared)

    loss = sum_diff_squared/m
    return loss

In [13]:
np.random.seed(123)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * .1
R = np.random.rand(n, n)
print(f"Expected loss for an experiment with random matrices: {compute_loss(X, Y, R):.4f}" ) 

Expected loss for an experiment with random matrices: 8.1866


#### Computing Gradient

In [14]:
def compute_gradient(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        g: a scalar value - gradient of the loss function L for given X, Y and R.
    '''
   
    m = X.shape[0]

    gradient = np.dot(X.transpose(),np.dot(X,R)-Y)*(2/m)
    
    return gradient

In [15]:
np.random.seed(123)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * .1
R = np.random.rand(n, n)
gradient = compute_gradient(X, Y, R)
print(f"First row of the gradient matrix: {gradient[0]}")

First row of the gradient matrix: [1.3498175  1.11264981 0.69626762 0.98468499 1.33828969]


#### Finding the optimal R with GD

In [16]:
def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003, verbose=True, compute_loss=compute_loss, compute_gradient=compute_gradient):
    '''
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        train_steps: positive int - describes how many steps will gradient descent algorithm do.
        learning_rate: positive float - describes how big steps will  gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n,n) - the projection matrix that minimizes the F norm ||X R -Y||^2
    '''
    np.random.seed(129)

    # the number of columns in X is the number of dimensions for a word vector (e.g. 300)
    # R is a square matrix with length equal to the number of dimensions in th  word embedding
    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if verbose and i % 25 == 0:
            print(f"loss at iteration {i} is: {compute_loss(X, Y, R):.4f}")
        
        gradient = compute_gradient(X,Y,R)

        R -= learning_rate * gradient
    return R

In [17]:
np.random.seed(129)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * .1
R = align_embeddings(X, Y)

loss at iteration 0 is: 3.7242
loss at iteration 25 is: 3.6283
loss at iteration 50 is: 3.5350
loss at iteration 75 is: 3.4442


#### Calculating the R transformation

In [20]:
R_train = align_embeddings(X_train, Y_train, train_steps=600, learning_rate=0.8)

loss at iteration 0 is: 963.0146
loss at iteration 25 is: 97.8292
loss at iteration 50 is: 26.8329
loss at iteration 75 is: 9.7893
loss at iteration 100 is: 4.3776
loss at iteration 125 is: 2.3281
loss at iteration 150 is: 1.4480
loss at iteration 175 is: 1.0338
loss at iteration 200 is: 0.8251
loss at iteration 225 is: 0.7145
loss at iteration 250 is: 0.6534
loss at iteration 275 is: 0.6185
loss at iteration 300 is: 0.5981
loss at iteration 325 is: 0.5858
loss at iteration 350 is: 0.5782
loss at iteration 375 is: 0.5735
loss at iteration 400 is: 0.5705
loss at iteration 425 is: 0.5686
loss at iteration 450 is: 0.5673
loss at iteration 475 is: 0.5665
loss at iteration 500 is: 0.5659
loss at iteration 525 is: 0.5655
loss at iteration 550 is: 0.5653
loss at iteration 575 is: 0.5651


#### Testing The translation KNN

In [21]:
def nearest_neighbor(v, candidates, k=1, cosine_similarity=cosine_similarity):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    similarity_l = []

    for row in candidates:
        cos_similarity = cosine_similarity(v, row)

        similarity_l.append(cos_similarity)

    sorted_ids = np.argsort(similarity_l)  
    
    sorted_ids = sorted_ids[::-1]
    
    k_idx = sorted_ids[:k]
    return k_idx

In [22]:
v = np.array([1, 0, 1])
candidates = np.array([[1, 0, 5], [-2, 5, 3], [2, 0, 1], [6, -9, 5], [9, 9, 9]])
print(candidates[nearest_neighbor(v, candidates, 3)])

[[2 0 1]
 [1 0 5]
 [9 9 9]]


#### Test Vocabulary

In [23]:
def test_vocabulary(X, Y, R, nearest_neighbor=nearest_neighbor):
    '''
    Input:
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the transform matrix which translates word embeddings from
        English to French word vector space.
    Output:
        accuracy: for the English to French capitals
    '''

    
    pred = np.dot(X,R)

    num_correct = 0

    for i in range(len(pred)):
        pred_idx = nearest_neighbor(pred[i],Y)

        if pred_idx == i:
            num_correct += 1

    accuracy = num_correct / len(pred)


    return accuracy

In [24]:
X_val, Y_val = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

In [25]:
acc = test_vocabulary(X_val, Y_val, R_train)  # this might take a minute or two
print(f"accuracy on test set is {acc:.3f}")

accuracy on test set is 0.559


#### reducing the search space with LSH

In [26]:
# get the positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
all_tweets = all_positive_tweets + all_negative_tweets

In [27]:
def get_document_embedding(tweet, en_embeddings, process_tweet=process_tweet):
    '''
    Input:
        - tweet: a string
        - en_embeddings: a dictionary of word embeddings
    Output:
        - doc_embedding: sum of all word embeddings in the tweet
    '''
    doc_embedding = np.zeros(300)

    
    processed_doc = process_tweet(tweet)
    for word in processed_doc:
        doc_embedding += en_embeddings.get(word,0)
    return doc_embedding

In [28]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"
tweet_embedding = get_document_embedding(custom_tweet, en_embeddings_subset)
tweet_embedding[-5:]

array([-0.00268555, -0.15378189, -0.55761719, -0.07216644, -0.32263184])

#### Storing all document vectors into a dictionary

In [29]:
def get_document_vecs(all_docs, en_embeddings, get_document_embedding=get_document_embedding):
    '''
    Input:
        - all_docs: list of strings - all tweets in our dataset.
        - en_embeddings: dictionary with words as the keys and their embeddings as the values.
    Output:
        - document_vec_matrix: matrix of tweet embeddings.
        - ind2Doc_dict: dictionary with indices of tweets in vecs as keys and their embeddings as the values.
    '''

    # the dictionary's key is an index (integer) that identifies a specific tweet
    # the value is the document embedding for that document
    ind2Doc_dict = {}

    document_vec_l = []

    for i, doc in enumerate(all_docs):

        doc_embedding = get_document_embedding(doc, en_embeddings)

        ind2Doc_dict[i] = doc_embedding

        document_vec_l.append(doc_embedding)


    document_vec_matrix = np.vstack(document_vec_l)

    return document_vec_matrix, ind2Doc_dict

In [30]:
document_vecs, ind2Tweet = get_document_vecs(all_tweets, en_embeddings_subset)

In [31]:

print(f"length of dictionary {len(ind2Tweet)}")
print(f"shape of document_vecs {document_vecs.shape}")

length of dictionary 10000
shape of document_vecs (10000, 300)


In [32]:
my_tweet = 'i am sad'
process_tweet(my_tweet)
tweet_embedding = get_document_embedding(my_tweet, en_embeddings_subset)

In [33]:
idx = np.argmax(cosine_similarity(document_vecs, tweet_embedding))
print(all_tweets[idx])

@hanbined sad pray for me :(((


#### The Most similar tweets with LSH

In [34]:
N_VECS = len(all_tweets)       # This many vectors.
N_DIMS = len(ind2Tweet[1])     # Vector dimensionality.
print(f"Number of vectors is {N_VECS} and each has {N_DIMS} dimensions.")

Number of vectors is 10000 and each has 300 dimensions.


In [35]:
# The number of planes. We use log2(625) to have ~16 vectors/bucket.
N_PLANES = 10
# Number of times to repeat the hashing to improve the search.
N_UNIVERSES = 25

In [36]:
np.random.seed(0)
planes_l = [np.random.normal(size=(N_DIMS, N_PLANES))
            for _ in range(N_UNIVERSES)]

In [37]:
def hash_value_of_vector(v, planes):
    """Create a hash for a vector; hash_id says which random hash to use.
    Input:
        - v:  vector of tweet. It's dimension is (1, N_DIMS)
        - planes: matrix of dimension (N_DIMS, N_PLANES) - the set of planes that divide up the region
    Output:
        - res: a number which is used as a hash for your vector

    """
    
    dot_product = np.dot(v,planes)

    sign_of_dot_product = np.sign(dot_product)

    
    h = sign_of_dot_product>=0

    h = np.squeeze(h)

    hash_value = 0

    n_planes = planes.shape[1]
    for i in range(n_planes):
        hash_value += np.power(2,i)*h[i]
        
  
    hash_value = int(hash_value)

    return hash_value

In [38]:
np.random.seed(0)
idx = 0
planes = planes_l[idx]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
print(f" The hash value for this vector,",
      f"and the set of planes at index {idx},",
      f"is {hash_value_of_vector(vec, planes)}")

 The hash value for this vector, and the set of planes at index 0, is 768


In [39]:
def make_hash_table(vecs, planes, hash_value_of_vector=hash_value_of_vector):
    """
    Input:
        - vecs: list of vectors to be hashed.
        - planes: the matrix of planes in a single "universe", with shape (embedding dimensions, number of planes).
    Output:
        - hash_table: dictionary - keys are hashes, values are lists of vectors (hash buckets)
        - id_table: dictionary - keys are hashes, values are list of vectors id's
                            (it's used to know which tweet corresponds to the hashed vector)
    """
    num_of_planes = planes.shape[1]

    
    num_buckets = 2**num_of_planes

   
    hash_table = {i: [] for i in range(num_buckets)}

   
    id_table = {i: [] for i in range(num_buckets)}

    for i, v in enumerate(vecs):
        h = hash_value_of_vector(v, planes)

        
        hash_table[h].append(v) 

        
        id_table[h].append(i) 

    return hash_table, id_table

In [40]:
planes = planes_l[0]  # get one 'universe' of planes to test the function
tmp_hash_table, tmp_id_table = make_hash_table(document_vecs, planes)

print(f"The hash table at key 0 has {len(tmp_hash_table[0])} document vectors")
print(f"The id table at key 0 has {len(tmp_id_table[0])} document indices")
print(f"The first 5 document indices stored at key 0 of id table are {tmp_id_table[0][0:5]}")

The hash table at key 0 has 3 document vectors
The id table at key 0 has 3 document indices
The first 5 document indices stored at key 0 of id table are [3276, 3281, 3282]


In [41]:
def create_hash_id_tables(n_universes):
    hash_tables = []
    id_tables = []
    for universe_id in range(n_universes):  # there are 25 hashes
        print('working on hash universe #:', universe_id)
        planes = planes_l[universe_id]
        hash_table, id_table = make_hash_table(document_vecs, planes)
        hash_tables.append(hash_table)
        id_tables.append(id_table)
    
    return hash_tables, id_tables

hash_tables, id_tables = create_hash_id_tables(N_UNIVERSES)

working on hash universe #: 0
working on hash universe #: 1
working on hash universe #: 2
working on hash universe #: 3
working on hash universe #: 4
working on hash universe #: 5
working on hash universe #: 6
working on hash universe #: 7
working on hash universe #: 8
working on hash universe #: 9
working on hash universe #: 10
working on hash universe #: 11
working on hash universe #: 12
working on hash universe #: 13
working on hash universe #: 14
working on hash universe #: 15
working on hash universe #: 16
working on hash universe #: 17
working on hash universe #: 18
working on hash universe #: 19
working on hash universe #: 20
working on hash universe #: 21
working on hash universe #: 22
working on hash universe #: 23
working on hash universe #: 24


In [42]:
def approximate_knn(doc_id, v, planes_l, hash_tables, id_tables, k=1, num_universes_to_use=25, hash_value_of_vector=hash_value_of_vector):
    """Search for k-NN using hashes."""
    
    vecs_to_consider_l = list()

    ids_to_consider_l = list()

    ids_to_consider_set = set()

    for universe_id in range(num_universes_to_use):

        planes = planes_l[universe_id]

        hash_value = hash_value_of_vector(v, planes)

        hash_table = hash_tables[universe_id]

        document_vectors_l = hash_table[hash_value]

        id_table = id_tables[universe_id]

        new_ids_to_consider = id_table[hash_value]

       
        for i, new_id in enumerate(new_ids_to_consider):
            
            if doc_id == new_id:
                continue

            if new_id not in ids_to_consider_set:
                document_vector_at_i = document_vectors_l[i]  # Get the embedding for the new_id
                vecs_to_consider_l.append(document_vector_at_i)  # Append the embedding to the list of vectors to consider

                ids_to_consider_l.append(new_id)

                ids_to_consider_set.add(new_id)  # Add to the set to avoid duplicates

        
    print("Fast considering %d vecs" % len(vecs_to_consider_l))

    vecs_to_consider_arr = np.array(vecs_to_consider_l)

    nearest_neighbor_idx_l = nearest_neighbor(v, vecs_to_consider_arr, k=k)

    
    nearest_neighbor_ids = [ids_to_consider_l[idx]
                            for idx in nearest_neighbor_idx_l]

    return nearest_neighbor_ids

In [43]:
doc_id = 0
doc_to_search = all_tweets[doc_id]
vec_to_search = document_vecs[doc_id]

In [44]:
nearest_neighbor_ids = approximate_knn(
    doc_id, vec_to_search, planes_l, hash_tables, id_tables, k=3, num_universes_to_use=5)

Fast considering 77 vecs


In [45]:
print(f"Nearest neighbors for document {doc_id}")
print(f"Document contents: {doc_to_search}")
print("")

for neighbor_id in nearest_neighbor_ids:
    print(f"Nearest neighbor at document id {neighbor_id}")
    print(f"document contents: {all_tweets[neighbor_id]}")

Nearest neighbors for document 0
Document contents: #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

Nearest neighbor at document id 51
document contents: #FollowFriday @France_Espana @reglisse_menthe @CCI_inter for being top engaged members in my community this week :)
Nearest neighbor at document id 2478
document contents: #ShareTheLove @oymgroup @musicartisthere for being top HighValue members this week :) @nataliavas http://t.co/IWSDMtcayt
Nearest neighbor at document id 105
document contents: #FollowFriday @straz_das @DCarsonCPA @GH813600 for being top engaged members in my community this week :)
